# Leto Demo Notebook

## Install all requirements

Move to root of repo to run code. 

In [2]:
cd ..

/root/aEye


In [2]:
!pip install -r requirements.txt

Keyring is skipped due to an exception: 'keyring.backends'
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
!pip install opencv-python-headless

## Imports

In [3]:
import boto3
import cv2
import logging
import sys
from aEye.video import Video
from aEye.labeler import Labeler
from aEye.auxiliary import Aux
from aEye.RealBasicVSR_reconstruction_module.builder import download_model
root = logging.getLogger()
root.setLevel(logging.INFO)
root.setLevel(logging.DEBUG)
root.setLevel(logging.ERROR)

/opt/conda/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370141920/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
/opt/conda/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


## Create objects to process original video into the correct sizing

In [5]:
aux = Aux()

In [6]:
labeler = Labeler()

---aEye Video Label Maker v0---


# Section 1: Trimming original video to 10 seconds. 

In [7]:
# video_list_s3 = aux.load_s3(bucket = 'leto-dish', prefix = 'original-videos/random-videos/')

In [8]:
# #print the metadata
# print(video_list_s3[0].get_meta_data())

From the metadata we can see that the original codec is h.264, and the video is 1080p

### Turn video into 10 second clip -- only performed once. 

In [9]:
# trimmed_s3 = process.add_label_trimming_start_duration(video_list_s3,0,10)

In [10]:
# aux.execute_label_and_write_local(trimmed_s3)

In [11]:
# aux.upload_s3(trimmed_s3, bucket = 'leto-dish', prefix = 'original-videos/random-videos/')

In [12]:
# aux.clean()

# Section 2: Downsampling trimmed video

In [13]:
video_list_s3 = aux.load_s3(bucket = 'leto-dish', prefix = 'original-videos/random-videos/')

In [14]:
print(video_list_s3)

[demo_10_second_clip.mp4]


In [17]:
#print the metadata
print(video_list_s3[0].extract_metadata())

{'streams': [{'index': 0, 'codec_name': 'h264', 'codec_long_name': 'H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10', 'profile': 'High', 'codec_type': 'video', 'codec_tag_string': 'avc1', 'codec_tag': '0x31637661', 'width': 1920, 'height': 1080, 'coded_width': 1920, 'coded_height': 1080, 'closed_captions': 0, 'film_grain': 0, 'has_b_frames': 2, 'pix_fmt': 'yuv420p', 'level': 40, 'color_range': 'tv', 'color_space': 'bt709', 'color_transfer': 'bt709', 'color_primaries': 'bt709', 'chroma_location': 'left', 'field_order': 'progressive', 'refs': 1, 'is_avc': 'true', 'nal_length_size': '4', 'id': '0x1', 'r_frame_rate': '25/1', 'avg_frame_rate': '25/1', 'time_base': '1/12800', 'start_pts': 0, 'start_time': '0.000000', 'duration_ts': 128000, 'duration': '10.000000', 'bit_rate': '79042', 'bits_per_raw_sample': '8', 'nb_frames': '250', 'extradata_size': 50, 'disposition': {'default': 1, 'dub': 0, 'original': 0, 'comment': 0, 'lyrics': 0, 'karaoke': 0, 'forced': 0, 'hearing_impaired': 0, 'visual_impair

From the metadata we can see that the original codec is h.264, and the video is 1080p

### Downsample the videos

In [21]:
downsampled_video = labeler.change_resolution(video_list_s3,"240p")

In [22]:
aux.execute_label_and_write_local(downsampled_video, "reduced_videos")

[resized_426x240_demo_10_second_clip.mp4]

In [24]:
aux.upload_s3(downsampled_video, bucket = 'leto-dish', prefix = 'reduced-videos/')

# Section 3: Reconstruction Video

Here we will use the realbasicvsr module to super resolve the video 4x

Setup Impoorts

In [31]:
PathToRepo = "/root/aEye/RealBasicVSR"
sys.path.append(PathToRepo)

In [33]:
conda update -n base -c defaults conda

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.5.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - conda


The following packages will be REMOVED:

  tqdm-4.42.1-py_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [34]:
conda install pytorch==1.7.1 torchvision==0.8.2 torchaudio==0.7.2 cudatoolkit=10.1 -c pytorch

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.5.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [35]:
!pip install openmim
!mim install mmcv-full

Keyring is skipped due to an exception: 'keyring.backends'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 1.2 MB/s eta 0:00:00ta 0:00:01
Keyring is skipped due to an exception: 'keyring.backends'
Looking in links: https://download.openmmlab.com/mmcv/dist/cu101/torch1.7.0/index.html


In [36]:
!pip install mmedit

Keyring is skipped due to an exception: 'keyring.backends'


Download the pretrained model

In [5]:
download_model("aEye/RealBasicVSR_reconstruction_module/RealBasicVSR_x4.pth","leto-dish","pretrained-models/RealBasicVSR_x4.pth")

Downloaded file to: aEye/RealBasicVSR_reconstruction_module/RealBasicVSR_x4.pth


In [ ]:
!python reconstruction_realbasicvsr.py /root/aEye/reduced_videos/resized_426x240_demo_10_second_clip.mp4 /root/aEye/reconstructed_videos/upsampled_demo.mp4 --fps=12.5

/opt/conda/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370141920/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
/opt/conda/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '
2023-07-11 19:27:34,803 - mmedit - INFO - load checkpoint from torchvision path: torchvision://vgg19
load checkpoint f

In [13]:
print("reconstructed via VSR")

reconstructed via VSR
